In [1]:
import torch

In [2]:
input_ids = torch.tensor([2,3,5,1])

In [3]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)

embedding_layer = torch.nn.Embedding(vocab_size , output_dim)


In [6]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [7]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [8]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


### Positional Embeddings

In [9]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size , output_dim)

In [15]:
with open("/Users/pandhari/LLM/STAGE_1/DATA_PREPARATION/TOKENIZATION/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [16]:
import tiktoken

In [17]:
from torch.utils.data import Dataset , DataLoader
import torch

class GPTDatasetV1(Dataset):

    def __init__(self , txt , tokenizer , max_length , stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt , allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0 , len(token_ids) - max_length , stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
        
    def __getitem__(self , idx):
        return self.input_ids[idx] , self.target_ids[idx]

In [18]:
def create_dataloader_v1(txt , batch_size = 4 , max_length = 256 , stride = 128
                      , shuffle = True , drop_last = True , num_workers = 0):
    
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create the dataset
    dataset = GPTDatasetV1(txt , tokenizer , max_length , stride)

    # Create the DataLoader
    dataloader = DataLoader(
        dataset , 
        batch_size = batch_size , 
        shuffle = shuffle , 
        drop_last = drop_last , 
        num_workers = num_workers
    )

    return dataloader

In [19]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text , batch_size=8 , max_length=max_length , 
    stride=max_length , shuffle=False
)

data_iter = iter(dataloader)
inputs , targets = next(data_iter)

In [20]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [23]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


In [24]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [25]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [26]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
